In [2]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [11]:
data_df = pd.read_pickle('./pickles/data_clean.pickle')
data_df

text
ama        tldr do whatever the fuck you want so youre te...
askreddit  victorias secret was originally supposed to be...
dankmemes  meme of the month seeding and nominations for ...
funny      but i wanted the robot ferret haha suck it you...
memes      redditmc is opening staff positions builders h...
science    am i reading this correctly to conclude that t...
worldnews  so was there any posts left after the purge ok...
wsb        after  green days in a row  wsb wtf where did ...

In [13]:
def replace_row(text):
    return re.sub('a{3,}', '', text)


replaced = lambda x: replace_row(x)
replaced

<function __main__.<lambda>(x)>

In [14]:
data_replaced = pd.DataFrame(data_df.text.apply(replaced))

In [15]:
data_replaced

text
ama        tldr do whatever the fuck you want so youre te...
askreddit  victorias secret was originally supposed to be...
dankmemes  meme of the month seeding and nominations for ...
funny      but i wanted the robot ferret haha suck it you...
memes      redditmc is opening staff positions builders h...
science    am i reading this correctly to conclude that t...
worldnews  so was there any posts left after the purge ok...
wsb        after  green days in a row  wsb wtf where did ...

In [16]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
# add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
#                   'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = TfidfVectorizer()
data_cvn = cvn.fit_transform(data_replaced.text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
data_dtmn.columns = data_df.index
data_dtmn

ama  askreddit  dankmemes  funny     memes   science  worldnews  \
aa        0.0   0.000104        0.0    0.0  0.000000  0.000000   0.000111   
aah       0.0   0.000012        0.0    0.0  0.000000  0.000162   0.000000   
aahh      0.0   0.000000        0.0    0.0  0.000000  0.000000   0.000000   
aahhhh    0.0   0.000014        0.0    0.0  0.000000  0.000000   0.000000   
aal       0.0   0.000000        0.0    0.0  0.000000  0.000000   0.000000   
...       ...        ...        ...    ...       ...       ...        ...   
𝙄𝙏        0.0   0.000014        0.0    0.0  0.000000  0.000000   0.000000   
𝙏𝙃𝙀       0.0   0.000028        0.0    0.0  0.000000  0.000000   0.000000   
𝙔𝙀𝙎       0.0   0.000014        0.0    0.0  0.000000  0.000000   0.000000   
𝚃𝚑𝚒𝚜      0.0   0.000000        0.0    0.0  0.000466  0.000000   0.000000   
𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎  0.0   0.000000        0.0    0.0  0.000466  0.000000   0.000000   

               wsb  
aa        0.000000  
aah       0.000000  
aahh      0.000306  
aahhhh    0.000000  
aal       0.002446  
...            ...  
𝙄𝙏        0.000000  
𝙏𝙃𝙀       0.000000  
𝙔𝙀𝙎       0.000000  
𝚃𝚑𝚒𝚜      0.000000  
𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎  0.000000  

[69209 rows x 8 columns]

In [6]:
# data_dtmn['index'] = data_dtmn.index
# for i, row in enumerate(data_dtmn['index']):
#     candidate = re.match('a{2,}', str(row))
#     if candidate != None:
#         data_dtmn.drop(row, inplace=True)
#     if row == 'ababababba':
#         data_dtmn.drop(row, inplace=True) 
# data_dtmn.drop(columns=['index'], inplace=True)
# data_dtmn[20:30]

wsb   science  ama  askreddit  worldnews  funny  dankmemes  \
abbie          0.0  0.000000  0.0   0.000014   0.000000    0.0   0.000000   
abbot          0.0  0.000036  0.0   0.000000   0.000000    0.0   0.000000   
abbott         0.0  0.000036  0.0   0.000000   0.000000    0.0   0.000000   
abbreviated    0.0  0.000014  0.0   0.000000   0.000000    0.0   0.000000   
abbreviates    0.0  0.000000  0.0   0.000000   0.000000    0.0   0.000000   
abbreviation   0.0  0.000011  0.0   0.000031   0.000049    0.0   0.000522   
abbreviations  0.0  0.000042  0.0   0.000012   0.000060    0.0   0.000000   
abby           0.0  0.000000  0.0   0.000112   0.000000    0.0   0.000000   
abbynormal     0.0  0.000000  0.0   0.000016   0.000000    0.0   0.000000   
abbys          0.0  0.000000  0.0   0.000000   0.000000    0.0   0.000000   

               memes   nosleep  psychology  politics  cscareerquestions  \
abbie            0.0  0.000000    0.000000  0.000000           0.000000   
abbot            0.0  0.000000    0.000000  0.000000           0.000000   
abbott           0.0  0.000000    0.000000  0.000000           0.000000   
abbreviated      0.0  0.000000    0.000433  0.000000           0.000042   
abbreviates      0.0  0.000000    0.000000  0.000000           0.000000   
abbreviation     0.0  0.000000    0.000000  0.000000           0.000034   
abbreviations    0.0  0.000000    0.000000  0.000000           0.000000   
abby             0.0  0.000913    0.000000  0.000023           0.000000   
abbynormal       0.0  0.000000    0.000000  0.000000           0.000000   
abbys            0.0  0.000000    0.000000  0.000000           0.000000   

               writingprompts  
abbie                0.000373  
abbot                0.000000  
abbott               0.000000  
abbreviated          0.000000  
abbreviates          0.000062  
abbreviation         0.000000  
abbreviations        0.000000  
abby                 0.001576  
abbynormal           0.000000  
abbys                0.000062

In [17]:
data_dtmn.transpose()

aa       aah      aahh    aahhhh       aal      aang  \
ama        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
askreddit  0.000104  0.000012  0.000000  0.000014  0.000000  0.001286   
dankmemes  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
funny      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
memes      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
science    0.000000  0.000162  0.000000  0.000000  0.000000  0.000000   
worldnews  0.000111  0.000000  0.000000  0.000000  0.000000  0.000000   
wsb        0.000000  0.000000  0.000306  0.000000  0.002446  0.000000   

              aangs      aapl     aaple  aardvark  ...       𝘽𝙊𝘽   𝘽𝙐𝙄𝙇𝘿𝙀𝙍  \
ama        0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
askreddit  0.000014  0.000000  0.000000  0.000000  ...  0.000028  0.000028   
dankmemes  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
funny      0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
memes      0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
science    0.000000  0.000000  0.000000  0.000194  ...  0.000000  0.000000   
worldnews  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
wsb        0.000000  0.040366  0.000306  0.000000  ...  0.000000  0.000000   

                𝘾𝘼𝙉       𝙁𝙄𝙓        𝙃𝙀        𝙄𝙏       𝙏𝙃𝙀       𝙔𝙀𝙎  \
ama        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
askreddit  0.000028  0.000014  0.000028  0.000014  0.000028  0.000014   
dankmemes  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
funny      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
memes      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
science    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
worldnews  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
wsb        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

               𝚃𝚑𝚒𝚜  𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎  
ama        0.000000  0.000000  
askreddit  0.000000  0.000000  
dankmemes  0.000000  0.000000  
funny      0.000000  0.000000  
memes      0.000466  0.000466  
science    0.000000  0.000000  
worldnews  0.000000  0.000000  
wsb        0.000000  0.000000  

[8 rows x 69209 columns]

In [18]:
from gensim import matutils, models
import scipy.sparse

In [19]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [20]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=10, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 22:55:10: using symmetric alpha at 0.1
INFO - 22:55:10: using symmetric eta at 0.1
INFO - 22:55:10: using serial LDA version on this node
INFO - 22:55:10: running online LDA training, 10 topics, 50 passes over the supplied corpus of 69209 documents, updating every 30000 documents, evaluating every ~69209 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 22:55:10: training LDA model using 15 processes
INFO - 22:55:10: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/69209, outstanding queue size 1
INFO - 22:55:10: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/69209, outstanding queue size 2
INFO - 22:55:10: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/69209, outstanding queue size 3
INFO - 22:55:10: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/69209, outstanding queue size 4
INFO - 22:55:10: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/69209, outstanding queue size 5
INFO - 22:

INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #4 = documents up to #10000/69209, outstanding queue size 5
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #5 = documents up to #12000/69209, outstanding queue size 6
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #6 = documents up to #14000/69209, outstanding queue size 6
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #7 = documents up to #16000/69209, outstanding queue size 6
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #8 = documents up to #18000/69209, outstanding queue size 7
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #9 = documents up to #20000/69209, outstanding queue size 5
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #10 = documents up to #22000/69209, outstanding queue size 6
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #11 = documents up to #24000/69209, outstanding queue size 6
INFO - 22:55:12: PROGRESS: pass 1, dispatched chunk #12 = documents up to #26000/69209, outstanding qu

INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #10 = documents up to #22000/69209, outstanding queue size 6
INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #11 = documents up to #24000/69209, outstanding queue size 6
INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #12 = documents up to #26000/69209, outstanding queue size 6
INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #13 = documents up to #28000/69209, outstanding queue size 6
INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #14 = documents up to #30000/69209, outstanding queue size 6
INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #15 = documents up to #32000/69209, outstanding queue size 6
INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #16 = documents up to #34000/69209, outstanding queue size 6
INFO - 22:55:14: PROGRESS: pass 2, dispatched chunk #17 = documents up to #36000/69209, outstanding queue size 4
INFO - 22:55:14: merging changes from 30000 documents into a model of 69209 documents
INFO - 22:

INFO - 22:55:16: PROGRESS: pass 3, dispatched chunk #17 = documents up to #36000/69209, outstanding queue size 5
INFO - 22:55:16: PROGRESS: pass 3, dispatched chunk #18 = documents up to #38000/69209, outstanding queue size 5
INFO - 22:55:16: merging changes from 30000 documents into a model of 69209 documents
INFO - 22:55:16: topic #4 (0.100): 0.000*"aahhhh" + 0.000*"aah" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aahh" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:55:16: topic #3 (0.100): 0.000*"aapl" + 0.000*"aahh" + 0.000*"aal" + 0.000*"aah" + 0.000*"aang" + 0.000*"aa" + 0.000*"aangs" + 0.000*"aahhhh" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:55:16: topic #8 (0.100): 0.000*"aah" + 0.000*"aal" + 0.000*"aahhhh" + 0.000*"aapl" + 0.000*"aa" + 0.000*"aahh" + 0.000*"aang" + 0.000*"aangs" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:55:16: topic #5 (0.100): 0.000*"aahhhh" + 0.000*"aahh" + 0.000*"aapl" + 0.000*"aa" + 0.000*"a

INFO - 22:55:17: topic #2 (0.100): 0.000*"aah" + 0.000*"aang" + 0.000*"aapl" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aahh" + 0.000*"aa" + 0.000*"aangs" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:55:17: topic #6 (0.100): 0.000*"aangs" + 0.000*"aang" + 0.000*"aal" + 0.000*"aahh" + 0.000*"aapl" + 0.000*"aahhhh" + 0.000*"aah" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:55:17: topic diff=0.000068, rho=0.158901
INFO - 22:55:17: PROGRESS: pass 4, dispatched chunk #19 = documents up to #40000/69209, outstanding queue size 5
INFO - 22:55:17: PROGRESS: pass 4, dispatched chunk #20 = documents up to #42000/69209, outstanding queue size 3
INFO - 22:55:17: PROGRESS: pass 4, dispatched chunk #21 = documents up to #44000/69209, outstanding queue size 3
INFO - 22:55:17: PROGRESS: pass 4, dispatched chunk #22 = documents up to #46000/69209, outstanding queue size 4
INFO - 22:55:17: PROGRESS: pass 4, dispatched chunk #23 = documents up to #48000/69209, outstanding queue s

...

INFO - 22:56:21: topic #7 (0.100): 0.000*"aahhhh" + 0.000*"aah" + 0.000*"aahh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:56:21: topic diff=0.000055, rho=0.110699
INFO - 22:56:21: PROGRESS: pass 46, dispatched chunk #19 = documents up to #40000/69209, outstanding queue size 5
INFO - 22:56:22: PROGRESS: pass 46, dispatched chunk #20 = documents up to #42000/69209, outstanding queue size 2
INFO - 22:56:22: PROGRESS: pass 46, dispatched chunk #21 = documents up to #44000/69209, outstanding queue size 3
INFO - 22:56:22: PROGRESS: pass 46, dispatched chunk #22 = documents up to #46000/69209, outstanding queue size 4
INFO - 22:56:22: PROGRESS: pass 46, dispatched chunk #23 = documents up to #48000/69209, outstanding queue size 5
INFO - 22:56:22: PROGRESS: pass 46, dispatched chunk #24 = documents up to #50000/69209, outstanding queue size 6
INFO - 22:56:22: PROGRESS: pass 46, dispatched chunk #25 = documents u

INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #24 = documents up to #50000/69209, outstanding queue size 5
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #25 = documents up to #52000/69209, outstanding queue size 5
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #26 = documents up to #54000/69209, outstanding queue size 5
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #27 = documents up to #56000/69209, outstanding queue size 5
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #28 = documents up to #58000/69209, outstanding queue size 5
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #29 = documents up to #60000/69209, outstanding queue size 5
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #30 = documents up to #62000/69209, outstanding queue size 5
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #31 = documents up to #64000/69209, outstanding queue size 4
INFO - 22:56:23: PROGRESS: pass 47, dispatched chunk #32 = documents up to #66000/69209,

INFO - 22:56:25: PROGRESS: pass 48, dispatched chunk #31 = documents up to #64000/69209, outstanding queue size 4
INFO - 22:56:25: PROGRESS: pass 48, dispatched chunk #32 = documents up to #66000/69209, outstanding queue size 5
INFO - 22:56:25: PROGRESS: pass 48, dispatched chunk #33 = documents up to #68000/69209, outstanding queue size 5
INFO - 22:56:25: merging changes from 30000 documents into a model of 69209 documents
INFO - 22:56:25: topic #1 (0.100): 0.000*"aahhhh" + 0.000*"aah" + 0.000*"aahh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:56:25: topic #0 (0.100): 0.002*"aapl" + 0.002*"aahh" + 0.002*"aal" + 0.002*"aahhhh" + 0.002*"aah" + 0.002*"aa" + 0.002*"aangs" + 0.002*"aang" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:56:25: topic #2 (0.100): 0.000*"aahhhh" + 0.000*"aah" + 0.000*"aahh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"po

INFO - 22:56:27: topic #1 (0.100): 0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:56:27: topic #7 (0.100): 0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:56:27: topic #2 (0.100): 0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:56:27: topic diff=0.000032, rho=0.108718
INFO - 22:56:27: merging changes from 7209 documents into a model of 69209 documents
INFO - 22:56:27: topic #2 (0.100): 0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"
INFO - 22:56:27: topic #4 (0.100): 0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.0

[(0,
  '0.002*"aapl" + 0.002*"aahh" + 0.002*"aal" + 0.002*"aahhhh" + 0.002*"aah" + 0.002*"aa" + 0.002*"aangs" + 0.002*"aang" + 0.000*"postmark" + 0.000*"postmarketing"'),
 (1,
  '0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"'),
 (2,
  '0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"'),
 (3,
  '0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"'),
 (4,
  '0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" + 0.000*"postmarketing"'),
 (5,
  '0.000*"aahh" + 0.000*"aah" + 0.000*"aahhhh" + 0.000*"aal" + 0.000*"aangs" + 0.000*"aapl" + 0.000*"aang" + 0.000*"aa" + 0.000*"postmark" +